In [27]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [26]:
MAX_LENGTH = 10

PAD_token = 0
SOS_token = 1
EOS_token = 2

In [22]:
# RNN编码器
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super().__init__()
        self.hidden_size = hidden_size # 隐藏态维度大小
        self.embedding = nn.Embedding(input_size, hidden_size) # 嵌入层
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True) # RNN单元
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, X):
        X = self.embedding(X)
        X = self.dropout(X)
        output, hidden = self.rnn(X)
        return output, hidden

In [25]:
encoder = EncoderRNN(input_size=10, hidden_size=5)
input_vector = torch.arange(10).unsqueeze(0) # （1,10，10）
output, hidden = encoder(input_vector) # output:(1,10,5) hidden(1,1,5)
print('输入向量的维度：',input_vector.size())
print('输出向量的维度：',output.size())
print('最终隐藏态的维度：',hidden.size())

输入向量的维度： torch.Size([1, 10])
输出向量的维度： torch.Size([1, 10, 5])
最终隐藏态的维度： torch.Size([1, 1, 5])


In [62]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long).fill_(
            SOS_token)  # Start of Sentence词元，用于表示开始生成一个句子
        decoder_hidden = encoder_hidden # 编码器隐藏态 作为 解码器隐藏态 (1,10,5)
        decoder_outputs = []
        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden = self.forward_step(
                decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output) # 输出追加到 decoder_outputs
            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1) # 最高概率的索引值
                decoder_input = topi.squeeze(-1).detach()


        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None

    def forward_step(self, x, hidden):
        x = self.embedding(x)
        x = F.relu(x)
        x, hidden = self.rnn(x, hidden) # x:(1,1,5) hidden(1,1,5)
        output = self.out(x)
        return output, hidden

In [63]:
decoder = DecoderRNN(hidden_size=5, output_size=10)
target_vector = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]) # (1,10) 模拟目标target
encoder_outputs, encoder_hidden = encoder(input_vector)
output, hidden, _ = decoder(encoder_outputs, encoder_hidden, target_vector)

topi: tensor([[[5]]])
topi: tensor([[[9]]])
topi: tensor([[[8]]])
topi: tensor([[[8]]])
topi: tensor([[[8]]])
topi: tensor([[[8]]])
topi: tensor([[[8]]])
topi: tensor([[[8]]])
topi: tensor([[[8]]])
topi: tensor([[[8]]])
torch.Size([1, 10, 10])


In [49]:
'''
前面部分代码实现了单独的Encoder2Decoder的Seq2Seq结构
主要思想是：Encoder的输出隐藏态作为Decoder的初始隐藏态
然后进行依次RNN单元的计算，每个RNNCell的输出就是该单元的预测
如果是进行强制学习的话，就是将target输出作为下一时刻输入
否则就是将当前时刻输出，作为下一时刻RNNCell的输入
这样逐Cell进行计算，计算出最终的预测序列
'''
print("输出向量的维度:", output.size())
print("最终隐藏状态的维度:", hidden.size())

输出向量的维度: torch.Size([1, 10, 10])
最终隐藏状态的维度: torch.Size([1, 1, 5])
